In [1]:
#install SDK
#!pip3 install --upgrade google-cloud-storage

In [3]:
from google.cloud import storage

In [7]:
#Create service account with permissions to read/write on Cloud storage bucket and get JSON
import platform
import os

platform=platform.system()
print(platform)
if platform == 'Windows':
    os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r'C:\Users\lafacero\Downloads\ai-vqc-fb43ef70f5ec.json'
elif platform == 'Linux':
    os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/home/rosario/Codice/GCP_Kubeflow_example/auth/ai-vqc-fb43ef70f5ec.json'
print(os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))

Linux
/home/rosario/Codice/GCP_Kubeflow_example/auth/ai-vqc-fb43ef70f5ec.json


In [29]:
#connect to Google Cloud Storage and retrieve data
#read data
PROJECT='ai-vqc'
BUCKET='ai-vqc-kubeflow-test'
PREFIX=''
FILENAME='test.h5'

storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET)
blob = bucket.blob(PREFIX+FILENAME)
blob.download_to_filename(FILENAME)

In [10]:
#write data
PROJECT='ai-vqc'
BUCKET='ai-vqc-kubeflow-output'
PREFIX=''
FILENAME='test.h5'

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET)
blob = bucket.blob(FILENAME)
blob.upload_from_filename(FILENAME)

In [14]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import KFold
#from sklearn.preprocessing import LabelEncoder
#from sklearn.pipeline import Pipeline
from keras.utils import to_categorical
import argparse
from pathlib import Path
from google.cloud import storage



Using TensorFlow backend.
/home/rosario/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rosario/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rosario/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rosario/.local/lib/python3.6/site-packages/tensorflow/py

In [22]:
PROJECT='ai-vqc'
INPUT_BUCKET='ai-vqc-kubeflow-test'
OUTPUT_BUCKET='ai-vqc-kubeflow-output'
PREFIX=''

def read_from_gcs(filename, path):
    storage_client = storage.Client()
    bucket = storage_client.bucket(INPUT_BUCKET)
    print(PREFIX+filename)
    blob = bucket.blob(PREFIX+filename)
    print(path+filename)
    blob.download_to_filename(path+filename)

def write_to_gcs(filename, path):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(OUTPUT_BUCKET)
    blob = bucket.blob(filename)
    blob.upload_from_filename(path+filename)
    print(path+filename, " written to ", OUTPUT_BUCKET, " bucket.")

def train(input_path, output):
    # load dataset
    print(input_path)
    dataframe = pandas.read_csv(input_path, error_bad_lines=False)
    dataframe.head()
    #dataset = dataframe.values
    #dataframe.head()

    X = dataframe.drop(columns=['totale_casi']).drop(columns=['data']).drop(columns=['stato']).drop(columns=['denominazione_regione'])
    Y = dataframe[['totale_casi']]
    Y_categorical=to_categorical(Y)

    model = Sequential()
    model.add(Dense(512, activation='relu', input_dim=X.shape[1]))
    model.add(Dense(256, activation='relu')) 
    model.add(Dense(128, activation='relu')) 
    model.add(Dense(Y_categorical.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    model.fit(X, Y_categorical, epochs=10)

    model.save(output)



In [26]:
input_filename='dpc-covid19-ita-regioni.csv'
output_filename='model.h5'
input_data=read_from_gcs(input_filename, '/home/rosario/Codice/GCP_Kubeflow_example/train/src/data/')
train(r'/home/rosario/Codice/GCP_Kubeflow_example/train/src/data/'+input_filename, r'/home/rosario/Codice/GCP_Kubeflow_example/train/src/data/'+output_filename)
write_to_gcs(output_filename, r'/home/rosario/Codice/GCP_Kubeflow_example/train/src/data/')

dpc-covid19-ita-regioni.csv
/home/rosario/Codice/GCP_Kubeflow_example/train/src/data/dpc-covid19-ita-regioni.csv
/home/rosario/Codice/GCP_Kubeflow_example/train/src/data/dpc-covid19-ita-regioni.csv
Epoch 1/10
294/294 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.0204                      
Epoch 2/10
294/294 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.2483
Epoch 3/10
294/294 [==============================] - 0s 882us/step - loss: nan - accuracy: 0.2483
Epoch 4/10
294/294 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.2483
Epoch 5/10
294/294 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.2483
Epoch 6/10
294/294 [==============================] - 0s 982us/step - loss: nan - accuracy: 0.2483
Epoch 7/10
294/294 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.2483
Epoch 8/10
294/294 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.2483
